In [1]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm
import pandas as pd

In [2]:
# config
data_config = 'Yelp'
config = {
    'Yelp':
    {
        'ds_name': 'Yelp',
        'ds_fmt': 'txt',
        'ds_rate': 20,
        'ds_toprate': 200,
        'ds_sep': ',',
    }
}

In [3]:
# loading dataset
dataset = pd.read_csv(f"datasets/{data_config}/raw/{data_config}_data_map.{config[data_config]['ds_fmt']}", sep=f"{config[data_config]['ds_sep']}", names=['uid', 'iid', 'freq'])

In [4]:
dataset.head()

,uid,iid,freq
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [5]:
item_freq = dict()

for index, eachline in tqdm(enumerate(dataset.itertuples(index=True))):
    if eachline.iid in item_freq.keys():
        # eachline.freq
        item_freq[eachline.iid] += 1
    elif (eachline.uid, eachline.iid) not in item_freq.keys():
        item_freq[eachline.iid] = 1

285608it [00:00, 537821.43it/s]


In [6]:
item_freq

{0: 56,
 1: 83,
 2: 17,
 3: 140,
 4: 11,
 5: 54,
 6: 4,
 7: 11,
 8: 63,
 9: 12,
 10: 81,
 11: 90,
 12: 32,
 13: 16,
 14: 173,
 15: 13,
 16: 43,
 17: 16,
 18: 37,
 19: 38,
 20: 41,
 21: 9,
 22: 4,
 23: 16,
 24: 29,
 25: 10,
 26: 12,
 27: 121,
 28: 16,
 29: 51,
 30: 79,
 31: 128,
 32: 16,
 33: 48,
 34: 147,
 35: 18,
 36: 12,
 37: 52,
 38: 113,
 39: 67,
 40: 33,
 41: 153,
 42: 33,
 43: 18,
 44: 74,
 45: 10,
 46: 16,
 47: 114,
 48: 16,
 49: 10,
 50: 16,
 51: 169,
 52: 40,
 53: 116,
 54: 21,
 55: 113,
 56: 21,
 57: 26,
 58: 79,
 59: 12,
 60: 151,
 61: 32,
 62: 88,
 63: 21,
 64: 53,
 65: 10,
 66: 50,
 67: 12,
 68: 135,
 69: 19,
 70: 13,
 71: 97,
 72: 15,
 73: 60,
 74: 11,
 75: 15,
 76: 19,
 77: 9,
 78: 39,
 79: 17,
 80: 28,
 81: 17,
 82: 58,
 83: 149,
 84: 16,
 85: 154,
 86: 26,
 87: 16,
 88: 14,
 89: 16,
 90: 13,
 91: 11,
 92: 17,
 93: 19,
 94: 139,
 95: 98,
 96: 24,
 97: 31,
 98: 133,
 99: 90,
 100: 23,
 101: 11,
 102: 36,
 103: 75,
 104: 9,
 105: 15,
 106: 11,
 107: 58,
 108: 12,
 109: 13

In [7]:
categories = pd.read_csv(f"datasets/{data_config}/raw/{data_config}_cat_map.txt", sep=f"{config[data_config]['ds_sep']}", names=['ItemID', 'Categories'])

In [8]:
cat_to_num = dict()
item_to_cat = dict()

for eachline in tqdm(categories.itertuples(index=True)):
    if eachline.ItemID not in item_to_cat:
        item_to_cat[eachline.ItemID] = [int(cat) for cat in eachline.Categories.split('|')]
    
    # if eachline.ItemID not in cat_to_num:
    for cat in eachline.Categories.split('|'):
        cat = int(cat)
        if cat not in cat_to_num:
            cat_to_num[cat] = item_freq[eachline.ItemID]
        elif cat in cat_to_num:
            cat_to_num[cat] += item_freq[eachline.ItemID]

16610it [00:00, 173519.94it/s]


In [9]:
sum_count = 0
for i in range(10, 21):
    sum_count += list(cat_to_num.values()).count(i)
print(sum_count)

62


In [10]:
outlier_cats = set()
for cat_id, cat_num in cat_to_num.items():
    if cat_num < 101:
        outlier_cats.add(cat_id)

In [11]:
len(outlier_cats)

305

In [12]:
outlier_items = set()
item_to_cat_verfied = dict()
for item_id, cat_list in item_to_cat.items():
    temp = []
    for cat_id in cat_list:
        if cat_id not in outlier_cats:
            temp.append(cat_id)
    if len(temp) != 0:
        item_to_cat_verfied[item_id] = temp
    else:
        outlier_items.add(item_id)

In [13]:
outlier_items

{1970,
 1979,
 4765,
 9156,
 9239,
 10174,
 10329,
 10425,
 11771,
 11960,
 12136,
 12178,
 12633,
 14114,
 14540,
 14568,
 14708,
 14902,
 15555,
 16126,
 16337}

In [14]:
len(item_to_cat_verfied)

16589

In [15]:
len(item_to_cat)

16610

In [ ]:
category_file = open(f"datasets/{data_config}/{data_config}_data_verified/{data_config}_cat_verified.txt", "w")

for item_id, categories_ids in item_to_cat_verfied.items():
    piped_categories = "|".join([str(cat) for cat in categories_ids])
    category_file.write(str(item_id) + "," + str(piped_categories) + "\n")

category_file.close()

In [ ]:
data_file = open(f"datasets/{data_config}/{data_config}_data_verified/{data_config}_data_verified.txt", "w")

for eachline in dataset.itertuples(index=True):
    if eachline.iid not in outlier_items:
        data_file.write(f"{eachline.uid},{eachline.iid},{eachline.freq}\n")
    else:
        print(eachline.iid)

data_file.close()